In [6]:
#import dependencies
import requests
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmaps
import os 
import pprint
import urllib

#import yelp_key
from config import yelp_key 

In [7]:
cities = ['Philadelphia','Dallas','Indianapolis','Atlanta','Seattle']
restaurant_ethnicities_df = pd.read_csv("../Resources/rest_enthinicity.csv")
ethnicities = [ethnicity for ethnicity in restaurant_ethnicities_df['ethnicity'] ]
ethnicities[ethnicities.index('indian')] = 'indpak'
ethnicities[ethnicities.index('new_american')] = 'newamerican'
ethnicities

['chinese',
 'french',
 'greek',
 'indpak',
 'italian',
 'japanese',
 'brazilian',
 'mexican',
 'newamerican',
 'soulfood',
 'thai']

In [9]:
responses = []
API_HOST = 'https://api.yelp.com/v3/businesses/search'
headers = {'Authorization': 'Bearer %s' % yelp_key}

for city in cities:
    for ethnicity in ethnicities:     
        params={'term':'restaurants','location':f'{city}', 'categories':f'{ethnicity}', 'offset':0}
        response = requests.get(API_HOST,headers=headers, params=params).json()
        response['ethnicity'] = f'{ethnicity}'
        response['city'] = f'{city}'
        responses.append(response)
        if response['total'] > 50:
            offset = 50
            while offset < response['total']:
                params={'term':'restaurants','location':f'{city}', 'categories':f'{ethnicity}', 'offset': offset}
                response = requests.get(API_HOST,headers=headers, params=params).json()
                response['ethnicity'] = f'{ethnicity}'
                response['city'] = f'{city}'
                responses.append(response)
                offset = offset + 50     

In [17]:
ethnicity_response_list = [responses[i]['ethnicity'] for i in range(len(responses))]
city_response_list = [responses[i]['city'] for i in range(len(responses))]

In [18]:
name = []
city = []
latitude = []
longitude = []
zipcode = []
ethnicity = []
price = []
rating = []

for i in range(len(responses)):
    for j in range(len(responses[i]['businesses'])):
        name.append(responses[i]["businesses"][j]["name"])
        city.append((responses[i]['city']))
        latitude.append(responses[i]["businesses"][j]["coordinates"]["latitude"])
        longitude.append(responses[i]["businesses"][j]["coordinates"]["longitude"])
        zipcode.append(responses[i]["businesses"][j]["location"]["zip_code"])
        ethnicity.append((responses[i]['ethnicity']))
        try:
            price.append(responses[i]["businesses"][j]["price"])
        except KeyError:
            price.append("N/A")
        rating.append(responses[i]["businesses"][j]["rating"])

In [20]:
rest_df = pd.DataFrame({
    "name": name,
    "city": city,
    "latitude": latitude,
    "longitude": longitude,
    "zipcode": zipcode,
    "ethnicity": ethnicity,
    "price": price,
    "rating": rating
})

rest_df

,name,city,latitude,longitude,zipcode,ethnicity,price,rating
0,Chubby Cattle,Philadelphia,39.954980,-75.156220,19107,chinese,N/A,4.5
1,Nan Zhou Hand Drawn Noodle House,Philadelphia,39.955401,-75.156901,19107,chinese,$,4.0
2,Dim Sum Garden,Philadelphia,39.955399,-75.156727,19107,chinese,$,4.0
3,Dim Sum House,Philadelphia,39.951885,-75.172466,19103,chinese,$$,4.0
4,Dan Dan,Philadelphia,39.950249,-75.167595,19102,chinese,$$,4.0
...,...,...,...,...,...,...,...,...
2646,Corner79,Seattle,47.686119,-122.344848,98103,thai,N/A,4.0
2647,Baitong on Wheels,Seattle,47.616933,-122.184873,98005,thai,N/A,5.0
2648,Olympic Express,Seattle,47.538540,-122.282829,98118,thai,$$,4.0
2649,Tangerine Thai,Seattle,47.672223,-122.354230,98103,thai,$$,3.5


In [7]:
# rest_df.to_csv("../Output/Data/_restaurants.csv", header = True, index = False)

,name,city,latitude,longitude,zipcode,ethnicity,price,rating
1556,Little Bangkok,Atlanta,33.815930,-84.352190,30324,chinese,$$,4.0
1557,Bao Bun Hibachi,Atlanta,33.801160,-84.414700,30318,chinese,$,4.5
1558,Urban Wu,Atlanta,33.846710,-84.374065,30305,chinese,$$,4.0
1559,Food Terminal,Atlanta,33.778829,-84.410034,30318,chinese,$$,4.5
1560,Hsu's Atlanta,Atlanta,33.759548,-84.386233,30303,chinese,$$,4.0
...,...,...,...,...,...,...,...,...
2040,Northlake Thai Cuisine,Atlanta,33.843272,-84.256376,30084,thai,$$,3.5
2041,Thai Cafe,Atlanta,33.879660,-84.467664,30339,thai,N/A,2.5
2042,T1 Bento,Atlanta,33.907766,-84.289732,30341,thai,$,4.0
2043,Tin Drum Asian Kitchen - Akers Mill Square,Atlanta,33.882804,-84.462033,30339,thai,$,2.5
